<a href="https://colab.research.google.com/github/MuneebBaig71/Chatbot/blob/main/WeatherBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain_community langchain_google_genai

In [ ]:
from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
os.environ['Weather_API'] = userdata.get('Weather_API')


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash',verbose=True)

In [ ]:
import requests
from langchain_core.tools import tool

@tool
def get_weather(city: str):
    """Gets the current weather in a given city.

    Args:
        city: The city to get the weather for.

    Returns:
        A string describing the weather in the given city.
    """
    api_key = os.environ['Weather_API']  # Use environment variable for API key
    url = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={city}&aqi=no"

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        weather = data["current"]["condition"]["text"]
        temperature = data["current"]["temp_c"]
        return f"The weather in {city} is {weather} with a temperature of {temperature}°C."
    else:
        return f"Unable to fetch weather data for {city}. Please check the city name or try again later."
llm_with_tools = llm.bind_tools([get_weather])

In [ ]:
tools_list=[get_weather]
model_with_tools = llm.bind_tools(tools_list)

In [ ]:
from langchain.agents import initialize_agent, AgentType

# List of tools
tools = [get_weather]

# Initialize the agent
agent = initialize_agent(
    tools=tools,                        # Provide the tools
    llm=llm,                            # LLM for fallback
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Agent type
    # verbose=True                        # Enable debugging output
)

In [ ]:
# Run the agent
agent.run("What is the weather in gujar khan?")

'The weather in Gujar Khan is Clear with a temperature of 7.5°C.'